In [2]:
#import ppa
import ppa
import numpy as np
import matplotlib.pyplot as plt
from PTMCMCSampler.PTMCMCSampler import PTSampler
from chainconsumer import ChainConsumer
from num2tex import num2tex
import json
import scipy.stats as ss
import corner
    
PSR_DICT_LIST = ppa.Load_All_Pulsar_Info()
PSR_NAME_LIST = list(PSR_DICT_LIST.keys())
PSR_DICT_LIST

Do not have mpi4py package.
Do not have acor package


{'J0437-4715': {'PSR': 'J0437-4715',
  'RAJ': '04:37:15.8961737',
  'DECJ': '-47:15:09.110714',
  'DTE_DM': 0.156,
  'PX': 6.37,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_20cm.txt'}},
 'J0613-0200': {'PSR': 'J0613-0200',
  'RAJ': '06:13:43.975901',
  'DECJ': '-02:00:47.23742',
  'DTE_DM': 1.024,
  'PX': 1.01,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0613-0200_ionFR_10cm.txt',
   '20cm': 

In [6]:
#=====================================================#
#     M + EFAC + EQUAD + Sred + Gamma                 #
#=====================================================#

Res = {}
k = 0
for i,psrn in enumerate(PSR_NAME_LIST):  
    res = {}
    for subset in ['10cm']:#psr.SUBSETS:

        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ],order=2,iono="subt",subset=subset,nfreqs=30)
        array = ppa.Array([psr])
        if array.NPSR != 0:
            _lnlike = array.Generate_Lnlike_Function(adm_signal='none')

            def lnlike(x):
                l10_EFAC , l10_EQUAD , l10_S0red  ,Gamma = np.array(x)
                return _lnlike( [l10_EFAC] , [l10_EQUAD] , [l10_S0red] , [Gamma] ,1, -22 , -8 )

            l10_EFAC_lp , l10_EFAC_sp = ppa.gen_uniform_lnprior(-2,2)
            l10_EQUAD_lp , l10_EQUAD_sp = ppa.gen_uniform_lnprior(-8,2)
            l10_S0red_lp , l10_S0red_sp = ppa.gen_uniform_lnprior(-8,2)
            Gamma_lp    , Gamma_sp = ppa.gen_uniform_lnprior(-3,3)
            
            def lnprior( x ):
                l10_EFAC , l10_EQUAD , l10_S0red , Gamma =  x

                LP_1 = l10_EFAC_lp(l10_EFAC)
                LP_2 = l10_EQUAD_lp(l10_EQUAD)
                LP_3 = l10_S0red_lp(l10_S0red)
                LP_4 = Gamma_lp(Gamma)

                return  np.sum([LP_1,LP_2,LP_3,LP_4])

            

            init = np.array([l10_EFAC_sp() , l10_EQUAD_sp() , l10_S0red_sp() , Gamma_sp() ])

            # sampler = PTSampler(len(init),lnlike,lnprior,cov = np.diag(np.ones(len(init)))*0.01,\
            #                     resume=False,outDir="SPA_Chain/red30_"+psr.PSR_NAME+"_"+subset,verbose=True)
            # sampler.sample(init,100000)

            chain  = np.loadtxt("SPA_Chain/red30_"+psrn+"_"+subset+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2


            med3 = np.median( chain[:,2])
            min3 = med3 - np.quantile( chain[:,2],0.16 )
            max3 = np.quantile( chain[:,2],0.84 ) - med3


            med4 = np.median( chain[:,3])
            min4 = med4 - np.quantile( chain[:,3],0.16 )
            max4 = np.quantile( chain[:,3],0.84 ) - med4



            res.update( { subset:(med1,med2,med3,med4)} )
            logVol = np.log( 4 * 10 * 10 * 6   )
            logz0 = lnlike([0,-7 , -7 , 0   ])
            logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
            
            # corner.corner(chain[:,[0,1,2,3]],labels=["l10EF" , "l10EQ" , "l10S0red","Gamma"]);
            # plt.savefig(psrn+".jpg")
            
            
            # with only psi12
            
            
            print( k+1,'&',psrn,"& %.2f"%np.log10(np.median(psr.DPA_ERR[0])),"& %.2f"%np.log10(np.std(psr.DPA[0])) ,\
                  "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f} $ & $ % .2f^{+%.2f}_{-%.2f}$ & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2,med3,max3,min3,med4,max4,min4),\
                    "& %.1f"%(logBF)+"\\\\" )
            
            k += 1

1 & J0437-4715 & -2.62 & -1.61 & $0.68^{+0.03}_{-0.03}$  & $-1.79^{+0.02}_{-0.02} $ & $  1.44^{+0.41}_{-0.72}$ & $-1.69^{+0.34}_{-0.19}$ & 43216.8\\
2 & J0613-0200 & -0.84 & -0.83 & $-0.19^{+0.05}_{-0.06}$  & $-1.02^{+0.05}_{-0.06} $ & $ -4.20^{+3.61}_{-2.78}$ & $-1.05^{+1.99}_{-1.36}$ & 17.0\\
3 & J0614-3329 & -1.12 & -1.21 & $-0.14^{+0.06}_{-0.09}$  & $-2.86^{+1.32}_{-3.40} $ & $ -4.27^{+3.51}_{-2.58}$ & $-1.27^{+1.53}_{-1.21}$ & 1.9\\
4 & J0711-6830 & -0.82 & -0.78 & $-0.13^{+0.02}_{-0.02}$  & $-1.24^{+0.05}_{-0.06} $ & $ -4.44^{+3.60}_{-2.54}$ & $-1.20^{+1.70}_{-1.24}$ & 27.7\\
5 & J1017-7156 & -1.21 & -1.09 & $-0.12^{+0.05}_{-0.05}$  & $-1.26^{+0.04}_{-0.04} $ & $ -4.37^{+3.47}_{-2.55}$ & $-1.00^{+1.89}_{-1.38}$ & 179.0\\
6 & J1022+1001 & -1.80 & -1.60 & $0.00^{+0.04}_{-0.03}$  & $-2.09^{+0.09}_{-0.17} $ & $  0.04^{+1.22}_{-1.48}$ & $-1.09^{+0.58}_{-0.58}$ & 29.3\\
7 & J1024-0719 & -1.47 & -1.41 & $-0.02^{+0.03}_{-0.05}$  & $-2.19^{+0.40}_{-3.86} $ & $ -4.51^{+3.57}_{-2.48}$ & $-1

In [7]:
with open("ppa/Parfile/spa_results.json",'w') as f:
    spa_results = json.dump(Res,f,indent=2)

In [8]:
#=====================================================#
#     M + EFAC + EQUAD                                #
#=====================================================#
k = 0
logw_all = []
for i,psrn in enumerate(PSR_NAME_LIST):  
    for subset in ['10cm']:#psr.SUBSETS:

        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ],order=2,iono="subt",subset=subset,nfreqs=0)
        array = ppa.Array([psr])
        if array.NPSR !=0 :
            _lnlike = array.Generate_Lnlike_Function(adm_signal='none')

            def lnlike(x):
                l10_EFAC , l10_EQUAD  = np.array(x)
                return _lnlike( [l10_EFAC] , [l10_EQUAD] , [-3.] , [0.] ,1., -22. , -8. )

            l10_EFAC_lp , l10_EFAC_sp = ppa.gen_uniform_lnprior(-2,2)
            l10_EQUAD_lp , l10_EQUAD_sp = ppa.gen_uniform_lnprior(-8,2)

            def lnprior( x ):
                l10_EFAC , l10_EQUAD  =  x

                LP_1 = l10_EFAC_lp(l10_EFAC)
                LP_2 = l10_EQUAD_lp(l10_EQUAD)

                return  np.sum([LP_1,LP_2])

            

            init = np.array([l10_EFAC_sp() , l10_EQUAD_sp() ])

            # sampler = PTSampler(len(init),lnlike,lnprior,cov = np.diag(np.ones(len(init)))*0.01,resume=False,outDir="SPA_Chain/white_"+psr.PSR_NAME+"_"+subset,verbose=True)
            # sampler.sample(init,100000)

            chain  = np.loadtxt("SPA_Chain/white_"+psrn+"_"+subset+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2



            logVol = np.log( 4 * 10   )
            logz0 = lnlike([0,-7.  ])
            logBF_white = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
            logw_all.append(logBF_white)

            corner.corner(chain[:,[0,1]],labels=["l10EF","l10EQ"]);
            plt.savefig(psrn+"_white.jpg")
            
            
            # with only psi12
            
            
            print( k+1,'&',psrn,"& $%.2f$"%np.log10(np.median(psr.DPA_ERR[0])),"& $%.2f$"%np.log10(np.std(psr.DPA[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f} $ &  & "%(med1,max1,min1,med2,max2,min2),"& %.1f"%(logBF_white)+"\\\\" )
            
            k += 1

Finished 10.00 percent in 3.786117 s Acceptance rate = 0.085189Adding DE jump with weight 20
Finished 99.00 percent in 37.428745 s Acceptance rate = 0.277364
Run Complete
1 & J0437-4715 & $-2.62$ & $-1.61$ & $0.69^{+0.03}_{-0.03}$  & $-1.77^{+0.02}_{-0.02} $ &  &  & 43187.1\\
Finished 10.00 percent in 2.598561 s Acceptance rate = 0.42388Adding DE jump with weight 20
Finished 99.00 percent in 25.914436 s Acceptance rate = 0.496677
Run Complete
2 & J0613-0200 & $-0.84$ & $-0.83$ & $-0.20^{+0.05}_{-0.06}$  & $-1.02^{+0.05}_{-0.06} $ &  &  & 13.8\\
Finished 10.00 percent in 2.171028 s Acceptance rate = 0.30233Adding DE jump with weight 20
Finished 99.00 percent in 20.274745 s Acceptance rate = 0.320747
Run Complete
3 & J0614-3329 & $-1.12$ & $-1.21$ & $-0.13^{+0.06}_{-0.09}$  & $-2.76^{+1.19}_{-3.58} $ &  &  & 1.6\\
Finished 10.00 percent in 2.559490 s Acceptance rate = 0.10911Adding DE jump with weight 20
Finished 99.00 percent in 27.063491 s Acceptance rate = 0.270768
Run Complete
4 & J0

# Old

In [2]:

#=====================================================#
#    Summarize                                        #
#=====================================================#
test = 0
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    try:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] ,subset="10cm",iono="subt",order=2,nfreqs=0)
        lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm")
        
        chain1  = np.loadtxt("SPA_Chain/m1_"+psrn+"_10cm/chain_1.txt",skiprows=5000)




        med1 = np.median( chain1[:,0])
        min1 = med1 - np.quantile( chain1[:,0],0.16 )
        max1 = np.quantile( chain1[:,0],0.84 ) - med1 


        med2 = np.median( chain1[:,1])
        min2 = med2 - np.quantile( chain1[:,1],0.16 )
        max2 = np.quantile( chain1[:,1],0.84 ) - med2 

        
        logz0 = lnlike([0,-np.inf])
        #test += logz0

        logBF1 = np.log( 0.02 * 0.01 * 0.1 ) - np.log( np.mean(np.exp( -( chain1[:,-4]-chain1[:,-4].max() ) ) ) )  + chain1[:,-4].max() - logz0



        psr_2 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=2 )
        psr_1 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=1 )
        psr_0 = ppa.Pulsar( PSR_DICT_LIST[ psrn ],subset="10cm",iono="subt" , order=0 )

        lnlike_2,lnprior,init_gen = psr_2.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        lnlike_1,lnprior,init_gen = psr_1.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        lnlike_0,lnprior,init_gen = psr_0.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,-7.99  ])
        
        dlnlike10 = lnlike_1([0,-np.inf ]) - lnlike_0([0,-np.inf ])
        dlnlike21 =  lnlike_2([0,-np.inf ]) - lnlike_1([0,-np.inf ])



        # print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)) ,
        #       "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
        #       "&%.1f&%.1f& %.1f"%(logBF1*np.log10(np.exp(1)),logBF2*np.log10(np.exp(1)),logBF*np.log10(np.exp(1)) )+"\\\\" )
        
        print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)),"&%.2f"%np.log10(np.std(psr.DPA[0])) ,
            "  & $% .2f^{+%.2f}_{-%.2f}$& $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
            "&${:.1f}$".format(num2tex(dlnlike10)) , "&${:.1f}$".format(num2tex(dlnlike21))  ,  "&${:.1f}$".format(num2tex(logBF1)) +"\\\\"  )
            #   "  &  %.1e  &  %.1e  &  %.1f "%(logBF1,logBF2 - logBF1,logBF3-logBF2 )+"\\\\" )
    except:
        pass

In [ ]:
for i,psrn in enumerate(PSR_NAME_LIST[:]): 
    try:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] ,subset="10cm",iono="subt",order=2)
        print(psr.NOBS)
    except:
        pass